### Task 2: Temporal Network Analysis
* Degree (in-degree/out-degree)
* Diameter
* Dyads and Reciprocity

A temporal network is goint to be analysed which consists of 678907 vertices and 4729035 edges, where each edge has time information associated with it. Some of the edges have the same source and target vertex, but are association with different timestamps. These timestamps are going to be used in this temporal netwrok analysis to asses and evaluate the evolution of grapg properties by analyzing the changes of the devised measuredby by exploring different time intervals and scales which define time windows to be used.

### <font color="darkgreen">Imports, configuation and preprocessing</font>

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import math
import igraph

#### Directed Graph used for temporal network analysis (networkx)

In [3]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G = nx.read_edgelist(wiki, create_using=nx.DiGraph())
wiki.close()

#### Directed Graph used for temporal network analysis (igraph)

In [135]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G_i = igraph.read(wiki) # used for certain functions absenst in networkx / are more efficient in igraph
wiki.close()

In [5]:
def create_dataframe(G_in):
    df_edge_in = pd.DataFrame(list(G_in.in_degree()), columns=['node', 'in edges'])
    df_edge_out = pd.DataFrame(list(G_in.out_degree()), columns=['node', 'out edges'])
    df_total = pd.merge(df_edge_in, df_edge_out, on='node')
    df_total.index = df_total.index + 1
    return df_total

In [6]:
wiki_df_par = create_dataframe(G_par)

In [7]:
wiki_df = create_dataframe(G)
wiki_df.head()

,node,in edges,out edges
1,1,28,88
2,6,10,0
3,8,62,6
4,9,1375,477
5,3,1068,451


### <font color="darkgreen">1. Temporal Network Analysis:</font> Degree distribution

### <font color="darkgreen">2. Temporal Network Analysis:</font> Diameter

### <font color="darkgreen">3. Temporal Network Analysis:</font> Dyads and Reciprocity